This notebook reads numpy arrays created by processMexicoData.ipynb. The processed data are then used to fine-tune the VGG16 model that has already been pre-trained on ImageNet data.

NB The notebook is set up for binary categories only ATM

In [ ]:
!pip install livelossplot

In [3]:
%load_ext autoreload
%autoreload 2

import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import load_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Input, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from livelossplot.inputs.keras import PlotLossesCallback

In [4]:
drive.mount('/content/drive')
sys.path.append('/content/drive/MyDrive/Colab Notebooks/')
import utils

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Read the numpy arrays containing Sentinel image snippets

states = ["Guanajuato", "Queretaro", "Sonora", "Yucatan"]
images_cat1 = []
images_cat0 = []

# The original datasets

for state in states:
  images_cat1.append(np.load(f'/content/drive/MyDrive/CAFO_data/Mexico/{state}_cat1_edited.npy'))
  images_cat0.append(np.load(f'/content/drive/MyDrive/CAFO_data/Mexico/{state}_cat0_edited.npy'))

images_cat1 = np.concatenate(images_cat1, axis=0)
images_cat0 = np.concatenate(images_cat0, axis=0)

print(f"There are {images_cat1.shape[0]} CAFO images")
print(f"There are {images_cat0.shape[0]} not-CAFO images")

There are 768 CAFO images
There are 821 not-CAFO images


In [6]:
# Create training, validation, and test datasets from those arrays

# Combine all the farm and not-farm images into a single array
images = np.concatenate([images_cat1, images_cat0], axis=0)

# Create an array of image labels
labels_cat1 = np.ones(len(images_cat1))
labels_cat0 = np.zeros(len(images_cat0))
labels = np.concatenate([labels_cat1, labels_cat0], axis=0)
labels = to_categorical(labels, num_classes=2)

# Randomly shuffle images and labels in unison
indices = np.arange(images.shape[0])
np.random.shuffle(indices) #Is there a default seed for this?
images = images[indices]
labels = labels[indices]

# Split into training + validation and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(
    images, labels, test_size=0.10, random_state=42)  # 10% for testing

# Split training + validation into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.22, random_state=42)  # ~20% for validation

# Keep copies of original data as the arrays can get changed by the data generators
X_train_original = np.copy(X_train)
X_val_original = np.copy(X_val)
X_test_original = np.copy(X_test)

In [7]:
# Set up data generators, augmenting training data but not validation + test data

train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    preprocessing_function=preprocess_input
    )

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
    )

In [8]:
# Create the model so as to fine-tune only the top 2 (non-convolutional) layers
# This is essentially extracting features and re-learning how to classify based on them

input_shape=X_train.shape[1:]
n_classes = 2
optimizer=Adam(learning_rate=0.001)
weights_file = '/content/drive/MyDrive/CAFO_models/VGG16_step1.keras'

# Load pretrained layers, excluding fully-connected layers
base_model = VGG16(include_top=False,
                  weights='imagenet',
                  input_shape=input_shape)

# Freeze pretrained layers
base_model.trainable = False

# Create new fully-connected layers for classification
top_model = base_model.output
top_model = Flatten(name="flatten")(top_model)
top_model = Dense(512, activation='relu')(top_model)
top_model = Dropout(0.5)(top_model)
top_model = Dense(256, activation='relu')(top_model)
top_model = Dropout(0.5)(top_model)
output_layer = Dense(n_classes, activation='softmax')(top_model)

# Combine the base (convolutional) and top (fully-connected) layers
model = Model(inputs=base_model.input, outputs=output_layer)

# If we have already done this step, can load saved weights from file
if weights_file is not None:
  print(f'Loading model from {weights_file}')
  model.load_weights(weights_file)

# Compile the model for training.
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Loading model from /content/drive/MyDrive/CAFO_models/VGG16_step1.keras


In [9]:
# Define early stopping + other callbacks. See
# machinelearningmastery.com/how-to-stop-training-deep-neural-networks-at-the-right-time-using-early-stopping/

es = EarlyStopping(
                  monitor='val_loss',
                  mode='min',
                  restore_best_weights=True,
                  min_delta=0.05,
                  patience=10,
                  verbose=2
                  )

mc = ModelCheckpoint(
                    '/content/drive/MyDrive/CAFO_models/VGG16_step1.keras',
                    monitor='val_accuracy',
                    mode='max',
                    verbose=1,
                    save_best_only=True
                    )

lr = ReduceLROnPlateau(
                      factor=0.5,
                      monitor='val_loss',
                      min_delta=0.05,
                      patience=5,
                      min_lr=1e-6,
                      verbose=1
                      )

pl = PlotLossesCallback()

In [10]:
# Fit the model, if we haven't reloaded a previously-fitted one

if weights_file is None:

  history = model.fit(
                    train_datagen.flow(X_train, y_train, batch_size=32),
                    validation_data=val_datagen.flow(X_val, y_val, batch_size=32),
                    epochs=50,
                    callbacks=[lr, es, mc, pl],
                    verbose=1
                    )

In [11]:
# Use the model to make predictions for the test data

# This uses the model at the final epoch of the previous step, which may not be
# the "best model" as defined by val accuracy and saved to file.
# I think this is OK because we're going to pick up fine-tuning from this step, just
# want to get an idea of how the model is performing that this stage.

y_prob, y_class = utils.get_predictions(model, X_test)
df = utils.collect_results(y_prob, y_test)

5/5 [==============================] - 8s 2s/step


In [12]:
# Generate sklearn classification report

print('\n', classification_report(y_test, y_class))


               precision    recall  f1-score   support

           0       0.86      0.87      0.86        77
           1       0.88      0.87      0.87        82

   micro avg       0.87      0.87      0.87       159
   macro avg       0.87      0.87      0.87       159
weighted avg       0.87      0.87      0.87       159
 samples avg       0.87      0.87      0.87       159



In [13]:
utils.plot_classified_images(X_test, df)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Distribution of probabilities

utils.probability_hist(df)

In [ ]:
# Fine-tune the model

# Specify new filename for best model
mc = ModelCheckpoint(
                    '/content/drive/MyDrive/CAFO_models/VGG16_step2.keras',
                    monitor='val_accuracy',
                    mode='max',
                    verbose=1,
                    save_best_only=True
                    )

# Reset the plots
pl = PlotLossesCallback()

# Set the top few convolutional layers to be trainable
# Do blocks not layers?
for layer in base_model.layers[-4:]:
    layer.trainable = True
    print(f"Layer {layer.name} is trainable")

# Compile the model with a lower learning rate
optimizer=Adam(learning_rate=0.001*0.2)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Fit the model
ft_history = model.fit(
                    train_datagen.flow(X_train, y_train, batch_size=32),
                    validation_data=val_datagen.flow(X_val, y_val, batch_size=32),
                    epochs=70, #base epochs + fine-tuning ones?
                    callbacks=[lr, mc, es, pl],
                    verbose=1
                    )

Layer block5_conv1 is trainable
Layer block5_conv2 is trainable
Layer block5_conv3 is trainable
Layer block5_pool is trainable
Epoch 1/70
34/35 [============================>.] - ETA: 3s - loss: 0.4871 - accuracy: 0.7959

In [ ]:
# This should ensure that we're using the "best model" from now on,
# not simply the model from the final epoch

model_ft = load_model('/content/drive/MyDrive/CAFO_models/VGG16_step2.keras')

y_prob_ft, y_class_ft = utils.get_predictions(model_ft, X_test)

df_ft = utils.collect_results(y_prob_ft, y_test)

In [ ]:
print('\n', classification_report(y_test, y_class_ft))

In [ ]:
utils.plot_classified_images(X_test, df_ft)

In [ ]:
utils.probability_hist(df_ft)